In [1]:
# default_exp markdown_merge

In [2]:
#export
from django.core.mail import *
from django.conf import *
from markdown import markdown
from email.headerregistry import Address
from time import sleep
import os

In [5]:
from nbdev.showdoc import *

In [7]:
#export
def get_addr(email, name=None):
    "Convert `email` and optional `name` into an email `Address` object"
    return Address(email if name is None else name, addr_spec=email)

In [8]:
#export
def md2email(subj, from_addr, to_addrs, md, conn=None, attach=None):
    "Create a multipart (markdown HTML and text) email"
    if not isinstance(to_addrs, (list,tuple)): to_addrs = [to_addrs]
    msg = EmailMultiAlternatives(subj, md, str(from_addr), [str(o) for o in to_addrs], connection=conn)
    msg.attach_alternative(markdown(md), "text/html")
    if attach is not None:
        if not isinstance(attach, (list,tuple)): attach = [attach]
        for att in attach: msg.attach_file(att)
    return msg

In [9]:
#export
class MarkdownMerge:
    "Send a templated email merge message formatted with Markdown"
    def __init__(self, addrs, from_addr, subj, msg, server_settings=None, inserts=None):
        if server_settings is None: os.environ['DJANGO_SETTINGS_MODULE'] = 'mail_settings'
        else:
            settings._wrapped=empty
            settings.configure(SECRET_KEY='XXX', **server_settings)
        self.addrs,self.from_addr,self.subj,self.msg,self.i = addrs,from_addr,subj,msg,0
        self.inserts = [{}]*len(addrs) if inserts is None else inserts

    def send_msgs(self, pause=0.5):
        "Send all unsent messages to `addrs` with `pause` secs between each send"
        with get_connection() as conn:
            while self.i < len(self.addrs):
                addr,insert = self.addrs[self.i],self.inserts[self.i]
                msg = self.msg.format(**insert)
                md2email(self.subj, self.from_addr, addr, md=msg, conn=conn).send()
                sleep(pause)
                self.i += 1
                if self.i%100==0: print(self.i)

    def set_test(self, test=True):
        "When `True`, just print the messages to the console, don't send them"
        backend = ('smtp','console')[test]
        settings.EMAIL_BACKEND = f'django.core.mail.backends.{backend}.EmailBackend'

    def reset(self):
        "Reset sent message list, so `send_msgs` will start from first message"
        self.i=0

Specify `from_addr` and `to_addrs` as either a string, or an `Address` object (created with `get_addr`). Note the `to_addrs` is a list.

In [46]:
from_addr = get_addr('from@example.com', 'Jeremy Howard')
to_addrs = [get_addr('to@example.com', 'Jeremy')]

Your message should be in [markdown](https://daringfireball.net/projects/markdown/syntax) format. It will be converted into a two part email, containing both a plain text and an HTML part, so recipients will see whatever format they're set as their preference for viewing mail. Anything in curly brackets `{}` will be replaced with the contents of the inserts dictionary for that address. If there are no bracketed variables to replace, then you don't need to pass any inserts.

In [10]:
msg = """
**Hello there!**

Here is your special message: *{special}*
"""

`inserts` is a list of dictionaries. For each dictionary, the keys should match the bracketed names in your email template, and the values will be filled in to those sections.

In [47]:
inserts = [{'special': "You are special."}]

In [48]:
ml = MarkdownMerge(to_addrs, from_addr, 'A message', msg=msg, inserts=inserts)

In the above example, we didn't specify any `server_settings`, so settings will be read from `mail_settings.py`. This is in Django settings format, so you can find complete docs for what can be provided in the [Django mail help](https://docs.djangoproject.com/en/2.2/topics/email/). A sample settings file is provided in the markdown_merge repo.

Alternatively, pass a dictionary to `server_settings` with the required keys, for instance:

In [ ]:
cfg = dict(EMAIL_HOST='smtp.fastmail.com', EMAIL_PORT=465,
    EMAIL_HOST_USER='aa@example.com', EMAIL_HOST_PASSWORD='XXX', EMAIL_USE_SSL=True)

In [14]:
show_doc(MarkdownMerge.set_test)

<h4 id="MarkdownMerge.set_test" class="doc_header"><code>MarkdownMerge.set_test</code><a href="https://github.com/jph00/markdown_merge/tree/master/__main__.py#L23" class="source_link" style="float:right">[source]</a></h4>

> <code>MarkdownMerge.set_test</code>(**`test`**=*`True`*)

When `True`, just print the messages to the console, don't send them

In [18]:
ml.set_test(True)

In [23]:
show_doc(MarkdownMerge.send_msgs)

<h4 id="MarkdownMerge.send_msgs" class="doc_header"><code>MarkdownMerge.send_msgs</code><a href="https://github.com/jph00/markdown_merge/tree/master/__main__.py#L12" class="source_link" style="float:right">[source]</a></h4>

> <code>MarkdownMerge.send_msgs</code>(**`pause`**=*`0.5`*)

Send all unsent messages to `addrs` with `pause` secs between each send

In [32]:
ml.send_msgs(pause=0.5)

Content-Type: multipart/alternative;
 boundary="===============7994339675932848898=="
MIME-Version: 1.0
Subject: A message
From: Jeremy Howard <from@example.com>
To: Jeremy <to1@example.com>
Date: Mon, 25 Nov 2019 06:11:11 -0000
Message-ID: <157466227172.47319.9791277819074450547@usf3>

--===============7994339675932848898==
Content-Type: text/plain; charset="utf-8"
MIME-Version: 1.0
Content-Transfer-Encoding: 7bit


**Hello there!**

Here is your special message: *You are special.*

--===============7994339675932848898==
Content-Type: text/html; charset="utf-8"
MIME-Version: 1.0
Content-Transfer-Encoding: 7bit

<p><strong>Hello there!</strong></p>
<p>Here is your special message: <em>You are special.</em></p>
--===============7994339675932848898==--

-------------------------------------------------------------------------------


Use `pause` to avoid sending too many messages too quickly; many SMTP servers restrict sending speed to avoid abuse. If you get an error during sending (e.g. "too many messages"), then wait an hour or so, then continue sending, using a larger `pause` value.

**NB**: You can just call `send_msgs` again when resending, since the successfully sent message count is saved, and those messages are not re-sent (unless you call `reset`). This includes test sends, therefore you should run reset after a test send.

In [29]:
show_doc(MarkdownMerge.reset)

<h4 id="MarkdownMerge.reset" class="doc_header"><code>MarkdownMerge.reset</code><a href="https://github.com/jph00/markdown_merge/tree/master/__main__.py#L28" class="source_link" style="float:right">[source]</a></h4>

> <code>MarkdownMerge.reset</code>()

Reset send message list, so `send_msgs` will start from first message

In [31]:
ml.reset()

## Utility functions

In [27]:
show_doc(md2email)

<h4 id="md2email" class="doc_header"><code>md2email</code><a href="https://github.com/jph00/markdown_merge/tree/master/__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>md2email</code>(**`subj`**, **`from_addr`**, **`to_addrs`**, **`md`**, **`conn`**=*`None`*, **`attach`**=*`None`*)

Create a multipart (markdown HTML and text) email

In [36]:
eml = md2email("hi", from_addr, to_addrs, msg)

The basic email body is the plain text message (note that the template variables in `{}` will be filled in by `MarkdownMerge`):

In [45]:
print(eml.body)


**Hello there!**

Here is your special message: *{special}*



Most email software is set up to display the HTML version:

In [44]:
from IPython.display import HTML
html,mimetype = eml.alternatives[0]
HTML(html)

In [28]:
show_doc(get_addr)

<h4 id="get_addr" class="doc_header"><code>get_addr</code><a href="https://github.com/jph00/markdown_merge/tree/master/__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>get_addr</code>(**`email`**, **`name`**=*`None`*)

Convert `email` and optional `name` into an email `Address` object

In [6]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_markdownmail.ipynb.
Converted 99_index.ipynb.
